In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
from causallearn.utils.cit import CIT

def check_sepset(X, Y, Z, sepsets):
    """
    Check if Y is in the separation set for nodes X and Z.
    """
    return Z in sepsets.get((X, Y), [])

def adjacent_nodes(node, adjacency):
    """
    List all nodes adjacent to the given node in the adjacency matrix.
    """
    return [idx for idx, connected in enumerate(adjacency[node]) if connected == 1]

def perform_pc_algorithm(dataframe):
    labels = dataframe.columns.tolist()
    label_indices = {label: idx for idx, label in enumerate(labels)}
    # Task 1
    num_vars = len(labels)
    data = dataframe.to_numpy()
    adjacency = -np.ones((num_vars, num_vars))
    np.fill_diagonal(adjacency, 0)

    cit_test = CIT(data, "fisherz")
    sepsets = {}
    # Task 2
    for size in range(num_vars - 2):
        updated = False
        for i in range(num_vars):
            for j in adjacent_nodes(i, adjacency):
                if j > i:
                    common_adj = set(adjacent_nodes(i, adjacency)) & set(adjacent_nodes(j, adjacency))
                    if len(common_adj) >= size:
                        for subset in combinations(common_adj, size):
                            if cit_test(i, j, list(subset)) > alpha:
                                adjacency[i][j] = adjacency[j][i] = 0
                                sepsets[(i, j)], sepsets[(j, i)] = list(subset), list(subset)
                                updated = True
        if not updated:
            break

    # Task 3 
    for i in range(num_vars):
        for j in adjacent_nodes(i, adjacency):
            for k in adjacent_nodes(j, adjacency):
                if k != i and adjacency[i][k] == 0 and not check_sepset(i, k, j, sepsets):
                    adjacency[i][j], adjacency[j][k] = 1, 1
                    adjacency[j][i], adjacency[k][j] = 0, 0

    # Task 4
    while True:
        changed = False
        for i in range(num_vars):
            for j in range(num_vars):
                if adjacency[i][j] == 1:  # Directed i -> j
                    for k in adjacent_nodes(j, adjacency):
                        if k != i and adjacency[j][k] == -1 and adjacency[i][k] == 0:
                            adjacency[j][k], adjacency[k][j] = 1, 0
                            changed = True
                        if adjacency[k][i] == 1 and adjacency[k][j] == -1:
                            adjacency[k][j], adjacency[j][k] = 1, 0
                            changed = True
        if not changed:
            break

    return adjacency

# Example usage:
np.random.seed(0)
n_samples = 2000
std_dev = np.sqrt(0.8)
X1, X3, X4 = [np.random.normal(0, std_dev, n_samples) for _ in range(3)]
X2 = 0.5 * X1 + np.random.normal(0, std_dev, n_samples)
X5 = 0.5 * X2 + 0.5 * X3 + 0.5 * X4 + np.random.normal(0, std_dev, n_samples)
X6 = 0.5 * X5 + np.random.normal(0, std_dev, n_samples)

df = pd.DataFrame({'X1': X1, 'X2': X2, 'X3': X3, 'X4': X4, 'X5': X5, 'X6': X6})
alpha = 0.05
adj_matrix = perform_pc_algorithm(df)
print(adj_matrix)

# Additional usage with real data
data_path = '/home/minghao.fu/workspace/Homework/ML803/week12/dataverse_files/galton-stata11.dta'
galton_data = pd.read_stata(data_path)
relevant_data = galton_data[['father', 'mother', 'gender', 'height']].copy()
relevant_data['gender_encoded'] = LabelEncoder().fit_transform(relevant_data['gender'])

data_for_pc = relevant_data[['father', 'mother', 'gender_encoded', 'height']]
adjacency_result = perform_pc_algorithm(data_for_pc)
print(adjacency_result)


[[ 0. -1. -1. -1. -1. -1.]
 [-1.  0. -1. -1. -1. -1.]
 [-1. -1.  0. -1. -1. -1.]
 [-1. -1. -1.  0. -1. -1.]
 [-1. -1. -1. -1.  0. -1.]
 [-1. -1. -1. -1. -1.  0.]]
[[ 0. -1. -1. -1.]
 [-1.  0. -1. -1.]
 [-1. -1.  0. -1.]
 [-1. -1. -1.  0.]]
